# 4. BeautifulSoup 활용
* 멜론 사이트 크롤링하기
* 제목, 가수, 앨범, 좋아요수 추출

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.melon.com/chart/index.htm'
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'    
}
res = requests.get(url,headers=headers)
res

<Response [200]>

In [3]:
res.text[:300]

'<!DOCTYPE html>\r\n<html lang="ko">\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<head>\r\n\t\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\r\n\t\r\n\r\n\t\r\n\r\n\t\r\n\r\n\t<title>멜론차트>TOP100>멜론</title>\r\n\t<meta name="keywor'

In [4]:
soup = BeautifulSoup(res.text)

## 노래제목 추출

In [14]:
songs = soup.select('.ellipsis.rank01')
title = []

for song in songs:
    title.append(song.find('a').get_text())
title[:10]

['STAY',
 '신호등',
 'Next Level',
 '바라만 본다',
 'OHAYO MY NIGHT',
 'Permission to Dance',
 'Weekend',
 '낙하 (with 아이유)',
 'Queendom',
 'Butter']

## 가수 이름 추출

In [6]:
songs = soup.select('.ellipsis.rank02')
singer = []

for song in songs:
    singer.append(song.find('a').get_text())
singer[:10]

['The Kid LAROI',
 '이무진',
 'aespa',
 'MSG워너비(M.O.M)',
 '디핵 (D-Hack)',
 '방탄소년단',
 '태연 (TAEYEON)',
 'AKMU (악뮤)',
 'Red Velvet (레드벨벳)',
 '방탄소년단']

## 앨범 추출

In [7]:
songs = soup.select('.ellipsis.rank03')
album = []

for song in songs:
    album.append(song.find('a').get_text())
album[:10]

['Stay',
 '신호등',
 'Next Level',
 'MSG워너비 1집',
 'OHAYO MY NIGHT',
 'Butter / Permission to Dance',
 'Weekend',
 'NEXT EPISODE',
 'Queendom - The 6th Mini Album',
 'Butter']

## 좋아요 수 추출

In [8]:
songs = soup.select('.button_etc.like')
params = ''

for song in songs:
    params += song['data-song-no']+ ','
params[:-1]

'33658563,33480898,33487342,33625988,32698101,33666269,33655994,33725775,33805289,33507137,33749109,33691794,33623210,33359317,33503722,32508053,33878014,32872978,33496587,33589488,33939317,33337651,33464805,30287019,33902950,33036660,33372781,33759183,33652821,33514925,33868791,33397561,33239419,33630420,33599821,33618271,33606505,33930393,33589260,33359725,33618864,33527705,32962258,33559180,8302665,33728859,31737197,31254010,3414749,33800786,33167063,32860397,33359309,30244931,33947769,33408084,33061995,32961718,1944399,33939315,33315511,33077234,33625989,8130796,32061975,33248758,33946475,33742378,32491274,33940932,33939313,32183386,33013877,33939312,33855085,33372788,30962526,32003395,32578498,33632153,31029291,33772329,33107649,32794652,3894276,33510307,33825315,32559782,32224272,33716984,5473462,31509376,32224166,33331004,32525311,33939316,31853557,33699116,33011180,32055419'

In [9]:
url2 = 'https://www.melon.com/commonlike/getSongLike.json?contsIds=' + params[:-1]

res = requests.get(url2, headers=headers)
res

<Response [200]>

In [10]:
songs = res.json()['contsLike']
like = []

for song in songs:
    like.append(song['SUMMCNT'])
like[:10]

[129404, 203876, 201176, 152723, 120887, 170468, 130977, 114785, 85556, 241445]

In [11]:
title[0], singer[0], album[0], like[0]

('STAY', 'The Kid LAROI', 'Stay', 129404)

In [12]:
# 제목, 가수, 앨범, 좋아요 dataframe 생성
df_data = {
    'title': title,
    'singer': singer,
    'album': album,
    'like': like
}

df = pd.DataFrame(data=df_data)
df

,title,singer,album,like
0,STAY,The Kid LAROI,Stay,129404
1,신호등,이무진,신호등,203876
2,Next Level,aespa,Next Level,201176
3,바라만 본다,MSG워너비(M.O.M),MSG워너비 1집,152723
4,OHAYO MY NIGHT,디핵 (D-Hack),OHAYO MY NIGHT,120887
...,...,...,...,...
95,한계,백예린 (Yerin Baek),선물,14182
96,오늘도 빛나는 너에게 (To You My Light) (Feat.이라온),마크툽 (MAKTUB),Red Moon : To You My Light,244652
97,고백,장범준,첫 번째 '고백',26095
98,나랑 같이 걸을래 (바른연애 길잡이 X 적재),적재,나랑 같이 걸을래 (바른연애 길잡이 X 적재),112757
